# Setup 

In [8]:
# Weights and Biases
#!pip install -q wandb
# Tensorflow
#!pip install -q tensorflow

In [15]:
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, Concatenate, Dense, BatchNormalization, LeakyReLU
from keras.activations import tanh
from tensorflow.keras.optimizers import Adam
from keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers.schedules import ExponentialDecay
import pandas as pd
import numpy as np

In [2]:
import wandb
from wandb.keras import WandbCallback
wandb.login(key='3cae81eb56be3190be5bb48c571e69933071df69')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: erlendrygg (avogadro). Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Erlend/.netrc


True

# Load data

In [22]:
file = "../data/complete data set/test data 100.csv"
df = pd.read_csv(file)
df.head(20)

val_set = 10
num_features = 4
time_steps = 20

train_y = df['Quote'][:-val_set].to_numpy()
val_y = df['Quote'][-val_set:].to_numpy()

df = df.drop(['Quote'], axis=1)

array = df.to_numpy()

train_x = [array[:-val_set][:,-time_steps:].reshape(len(array[:-val_set]),time_steps,1), array[:-val_set][:,:num_features]]
val_x = [array[-val_set:][:,-time_steps:].reshape(len(array[-val_set:]),time_steps,1), array[-val_set:][:,:num_features]]

print(train_x[0].shape)
print(train_x[1].shape)
print(val_x[0].shape)
print(val_x[1].shape)

(90, 20, 1)
(90, 4)
(10, 20, 1)
(10, 4)


# Model construction

In [31]:
def create_model(config):
    '''Builds an LSTM-MLP model of minimum 2 layers sequentially from a given config dictionary'''
    underlying_history = Input((config.LSTM_timesteps,1))
    bs_vars = Input((config.Num_features,))

    model = Sequential()

    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        input_shape = (config.LSTM_timesteps, 1),
        return_sequences = True
    ))

    for _ in range(config.LSTM_layers - 2):
        model.add(LSTM(
            units = config.LSTM_units,
            activation = tanh,
            return_sequences = True
        ))
    
    model.add(LSTM(
        units = config.LSTM_units,
        activation = tanh,
        return_sequences = False
    ))

    layers = Concatenate()([model(underlying_history), bs_vars])
    
    for _ in range(config.MLP_layers - 1):
        layers = Dense(config.MLP_units)(layers)
        layers = BatchNormalization(momentum=config.Bn_momentum)(layers)
        layers = LeakyReLU()(layers)

    #create the output layer
    output = Dense(1, activation='relu')(layers)

    #create exponentially decaying learning rate schedule
    lr_schedule = ExponentialDecay(
        initial_learning_rate = config.Lr,
        decay_steps = int(len(train_x)/config.Minibatch_size),
        decay_rate=config.Lr_decay
    )

    #compile the model
    model = Model(inputs=[underlying_history, bs_vars], outputs=output)
    model.compile(loss='mse', optimizer=Adam(learning_rate=lr_schedule))

    return model

# Hyperparameter search setup

In [48]:
# Configuring the sweep hyperparameter search space
sweep_configuration = {
    'method': 'random',
    'name': 'LSTM-MLP v.1.0 test data',
    'metric': {
        'goal': 'minimize', 
        'name': 'val_loss'
		},
    'parameters': {
        'LSTM_units': {
            'values': [32, 64, 96, 128]},
        'MLP_units': {
            'values': [32, 64, 96, 128]},
        'LSTM_timesteps': {
            'values': [20]},
        'LSTM_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 4},
        'MLP_layers': {
            'distribution': 'int_uniform',
            'max': 6, 'min': 4},
        'Bn_momentum': {
            'distribution': 'uniform',
            'max': 0.4, 'min': 0},
        'Lr': {
            'distribution': 'uniform',
            'max': 0.005, 'min': 0.0005},
        'Lr_decay': {
            'distribution': 'uniform',
            'max': 1, 'min': 0.9},        
        'Minibatch_size': {
            'values': [1024, 2048, 4096]},
        'Min_delta': {
            'value': 1},
        'Patience': {
            'value': 20},
        'Num_features': {
            'value': 4},
    }
}

# Initialize sweep and creating sweepID

# If new sweep, uncomment the line below and comment the line after it
#sweep_id = wandb.sweep(sweep=sweep_configuration, project='Deep learning for option pricing - test area') 
sweep_id = 'zyx7ephh'

Create sweep with ID: zyx7ephh
Sweep URL: https://wandb.ai/avogadro/Deep%20learning%20for%20option%20pricing%20-%20test%20area/sweeps/zyx7ephh


# Run hyperparameter search

In [33]:
def trainer(train_x = train_x, train_y = train_y, val_x = val_x, val_y = val_y, config = None, project = None):
    # Initialize a new wandb run
    with wandb.init(config=config, project = project):

        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config

        model = create_model(config)

        early_stopping = EarlyStopping(
            monitor='val_loss',
            mode='min',
            min_delta = config.Min_delta,
            patience = config.Patience,
        )
        
        wandb_callback = WandbCallback(
            monitor='val_loss',
            mode='min',
            save_model=False
        )
        
        model.fit(
            train_x,
            train_y,
            batch_size = config.Minibatch_size,
            validation_data = (val_x, val_y),
            epochs = 1000,
            callbacks = [early_stopping, wandb_callback] 
        )

## Run full sweep

In [49]:
wandb.agent(sweep_id=sweep_id, function=trainer, project='Deep learning for option pricing - test area', count = 10)

wandb: Agent Starting Run: x3u4w1i3 with config:
wandb: 	Bn_momentum: 0.062356576813404674
wandb: 	LSTM_layers: 4
wandb: 	LSTM_timesteps: 20
wandb: 	LSTM_units: 64
wandb: 	Lr: 0.003524075563119535
wandb: 	Lr_decay: 0.9754804861224868
wandb: 	MLP_layers: 5
wandb: 	MLP_units: 96
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 2048
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/1000
1/1 [==============================] - 8s 8s/step - loss: 0.2812 - val_loss: nan
Epoch 2/1000
1/1 [==============================] - 0s 85ms/step - loss: nan - val_loss: nan
Epoch 3/1000
1/1 [==============================] - 0s 82ms/step - loss: nan - val_loss: nan
Epoch 4/1000
1/1 [==============================] - 0s 87ms/step - loss: nan - val_loss: nan
Epoch 5/1000
1/1 [==============================] - 0s 105ms/step - loss: nan - val_loss: nan
Epoch 6/1000
1/1 [==============================] - 0s 106ms/step - loss: nan - val_loss: nan
Epoch 7/1000
1/1 [==============================] - 0s 101ms/step - loss: nan - val_loss: nan
Epoch 8/1000
1/1 [==============================] - 0s 89ms/step - loss: nan - val_loss: nan
Epoch 9/1000
1/1 [==============================] - 0s 103ms/step - loss: nan - val_loss: nan
Epoch 10/1000
1/1 [==============================] - 0s 85ms/step - loss: nan - val_loss: nan
Epoch 11/1000
1/1 [==============================] - 0s 84ms/ste

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,▁
epoch,19
loss,nan
val_loss,nan


wandb: Agent Starting Run: 68v97b2o with config:
wandb: 	Bn_momentum: 0.2948765596214656
wandb: 	LSTM_layers: 4
wandb: 	LSTM_timesteps: 20
wandb: 	LSTM_units: 128
wandb: 	Lr: 0.004362157443810796
wandb: 	Lr_decay: 0.9505828085760964
wandb: 	MLP_layers: 6
wandb: 	MLP_units: 128
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 1024
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/1000
1/1 [==============================] - 8s 8s/step - loss: 0.5141 - val_loss: nan
Epoch 2/1000
1/1 [==============================] - 0s 146ms/step - loss: nan - val_loss: nan
Epoch 3/1000
1/1 [==============================] - 0s 150ms/step - loss: nan - val_loss: nan
Epoch 4/1000
1/1 [==============================] - 0s 155ms/step - loss: nan - val_loss: nan
Epoch 5/1000
1/1 [==============================] - 0s 159ms/step - loss: nan - val_loss: nan
Epoch 6/1000
1/1 [==============================] - 0s 163ms/step - loss: nan - val_loss: nan
Epoch 7/1000
1/1 [==============================] - 0s 174ms/step - loss: nan - val_loss: nan
Epoch 8/1000
1/1 [==============================] - 0s 159ms/step - loss: nan - val_loss: nan
Epoch 9/1000
1/1 [==============================] - 0s 149ms/step - loss: nan - val_loss: nan
Epoch 10/1000
1/1 [==============================] - 0s 144ms/step - loss: nan - val_loss: nan
Epoch 11/1000
1/1 [==============================] - 0s 142

...

Epoch 1/1000
1/1 [==============================] - 8s 8s/step - loss: 0.5241 - val_loss: nan
Epoch 2/1000
1/1 [==============================] - 0s 128ms/step - loss: nan - val_loss: nan
Epoch 3/1000
1/1 [==============================] - 0s 122ms/step - loss: nan - val_loss: nan
Epoch 4/1000
1/1 [==============================] - 0s 122ms/step - loss: nan - val_loss: nan
Epoch 5/1000
1/1 [==============================] - 0s 126ms/step - loss: nan - val_loss: nan
Epoch 6/1000
1/1 [==============================] - 0s 180ms/step - loss: nan - val_loss: nan
Epoch 7/1000
1/1 [==============================] - 0s 153ms/step - loss: nan - val_loss: nan
Epoch 8/1000
1/1 [==============================] - 0s 153ms/step - loss: nan - val_loss: nan
Epoch 9/1000
1/1 [==============================] - 0s 123ms/step - loss: nan - val_loss: nan
Epoch 10/1000
1/1 [==============================] - 0s 131ms/step - loss: nan - val_loss: nan
Epoch 11/1000
1/1 [==============================] - 0s 122

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,▁
epoch,19
loss,nan
val_loss,nan


wandb: Agent Starting Run: dhuwafe2 with config:
wandb: 	Bn_momentum: 0.1590955092702703
wandb: 	LSTM_layers: 5
wandb: 	LSTM_timesteps: 20
wandb: 	LSTM_units: 96
wandb: 	Lr: 0.0021369819519819885
wandb: 	Lr_decay: 0.9937173294166136
wandb: 	MLP_layers: 6
wandb: 	MLP_units: 128
wandb: 	Min_delta: 1
wandb: 	Minibatch_size: 4096
wandb: 	Num_features: 4
wandb: 	Patience: 20
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Epoch 1/1000
1/1 [==============================] - 9s 9s/step - loss: 0.3029 - val_loss: nan
Epoch 2/1000
1/1 [==============================] - 0s 156ms/step - loss: nan - val_loss: nan
Epoch 3/1000
1/1 [==============================] - 0s 154ms/step - loss: nan - val_loss: nan
Epoch 4/1000
1/1 [==============================] - 0s 157ms/step - loss: nan - val_loss: nan
Epoch 5/1000
1/1 [==============================] - 0s 162ms/step - loss: nan - val_loss: nan
Epoch 6/1000
1/1 [==============================] - 0s 162ms/step - loss: nan - val_loss: nan
Epoch 7/1000
1/1 [==============================] - 0s 153ms/step - loss: nan - val_loss: nan
Epoch 8/1000
1/1 [==============================] - 0s 189ms/step - loss: nan - val_loss: nan
Epoch 9/1000
1/1 [==============================] - 0s 166ms/step - loss: nan - val_loss: nan
Epoch 10/1000
1/1 [==============================] - 0s 180ms/step - loss: nan - val_loss: nan
Epoch 11/1000
1/1 [==============================] - 0s 160

epoch,▁▁▂▂▂▃▃▄▄▄▅▅▅▆▆▇▇▇██
loss,▁
epoch,19
loss,nan
val_loss,nan


## Single run

In [39]:
config = {
    'LSTM_units': 4,
    'MLP_units': 100,
    'LSTM_timesteps': 20,
    'LSTM_layers': 4,
    'MLP_layers': 4,
    'Bn_momentum': 0.9,
    'Lr': 1e-4,
    'Lr_decay': 1,
    'Minibatch_size': 10,
    'Min_delta': 0.1,
    'Patience': 20,
    'Num_features': 4,
    'Architecture': 'LSTM-MLP v.0.1',
    'Dataset': 'Test data',
}
trainer(config = config, project = 'Deep learning for option pricing - test area')

Epoch 1/1000
9/9 [==============================] - 6s 184ms/step - loss: 0.3710 - val_loss: 0.3597
Epoch 2/1000
9/9 [==============================] - 0s 16ms/step - loss: 0.2466 - val_loss: 0.3527
Epoch 3/1000
9/9 [==============================] - 0s 17ms/step - loss: 0.2666 - val_loss: 0.3371
Epoch 4/1000
9/9 [==============================] - 0s 17ms/step - loss: 0.2639 - val_loss: 0.3746
Epoch 5/1000
9/9 [==============================] - 0s 18ms/step - loss: 0.2433 - val_loss: 0.3886
Epoch 6/1000
9/9 [==============================] - 0s 18ms/step - loss: 0.2405 - val_loss: 0.4096
Epoch 7/1000
9/9 [==============================] - 0s 17ms/step - loss: 0.2357 - val_loss: 0.4250
Epoch 8/1000
9/9 [==============================] - 0s 18ms/step - loss: 0.2189 - val_loss: 0.4539
Epoch 9/1000
9/9 [==============================] - 0s 17ms/step - loss: 0.2506 - val_loss: 0.4578
Epoch 10/1000
9/9 [==============================] - 0s 17ms/step - loss: 0.2681 - val_loss: 0.4731
Epoch 11

epoch,▁▁▂▂▂▃▃▃▄▄▅▅▅▆▆▆▇▇▇██
loss,█▃▄▃▃▂▂▂▃▄▂▂▃▁▂▁▁▃▁▁▃
val_loss,▂▂▁▃▄▅▆▇▇██▆▆▆▆▅▆▆▅▅▄
best_epoch,2
best_val_loss,0.33709
epoch,20
loss,0.2427
val_loss,0.39757
